In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST data
(x_train, _), (_, _) = mnist.load_data()

# Normalize images to the range [-1, 1] for GANs
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension

# Show a sample image
plt.imshow(x_train[0, :, :, 0], cmap='gray')
plt.show()


In [ ]:
from tensorflow.keras import layers

# Generator Model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, input_dim=100))
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2D(1, kernel_size=3, activation='tanh', padding='same'))
    
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(28, 28, 1)))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model


In [ ]:
# Compile the Discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile the combined model (Generator + Discriminator)
generator = build_generator()
discriminator.trainable = False

# Create the combined model (Generator + Discriminator)
z = layers.Input(shape=(100,))
fake_image = generator(z)
validity = discriminator(fake_image)
combined = tf.keras.models.Model(z, validity)
combined.compile(optimizer='adam', loss='binary_crossentropy')

# Train the GAN
epochs = 10000
batch_size = 64
half_batch = batch_size // 2

for epoch in range(epochs):
    # Train Discriminator with real images
    idx = np.random.randint(0, x_train.shape[0], half_batch)
    real_images = x_train[idx]
    
    # Generate fake images
    noise = np.random.normal(0, 1, (half_batch, 100))
    fake_images = generator.predict(noise)
    
    # Train the Discriminator (real images = 1, fake images = 0)
    d_loss_real = discriminator.train_on_batch(real_images, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train Generator (wants the Discriminator to classify as real)
    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Print progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")
    
    # Save generated images every 1000 epochs
    if epoch % 1000 == 0:
        noise = np.random.normal(0, 1, (25, 100))
        gen_images = generator.predict(noise)
        gen_images = 0.5 * gen_images + 0.5  # Rescale to [0, 1]
        
        fig, axs = plt.subplots(5, 5, figsize=(5, 5))
        count = 0
        for i in range(5):
            for j in range(5):
                axs[i, j].imshow(gen_images[count, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                count += 1
        plt.show()
